<a href="https://colab.research.google.com/github/AryanGupta0419/PytorchTutorial/blob/main/DL_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/samuelcortinhas/house-price-prediction-seattle'
od.download(dataset_url)

Dataset URL: https://www.kaggle.com/datasets/samuelcortinhas/house-price-prediction-seattle


100%|██████████| 25.9k/25.9k [00:00<00:00, 22.0MB/s]

IMPORTING DATASET

In [ ]:
train_path = "/content/house-price-prediction-seattle/train.csv"
test_path = "/content/house-price-prediction-seattle/test.csv"

test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)

df = pd.concat([train_df, test_df], axis=0)

def normalizecol(col):
  mean = df[col].mean()
  std = df[col].std()
  df[col] = (df[col] - mean)/std




feature = df.columns[:-1]
target = df.columns[-1]


df.dropna(inplace=True)
pricemean = df['price'].mean()
pricestd = df['price'].std()


df = df.drop(['size_units','lot_size_units','zip_code'], axis = 1)

for col in df.columns:
  normalizecol(col)



x= df.drop('price', axis = 1)
y=df['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

df.head()



,beds,baths,size,lot_size,price
0,-0.11679,0.192480,0.735757,0.774963,-0.260002
1,0.73359,-0.302166,0.353275,-1.435151,-0.132555
2,0.73359,0.687126,0.134713,-0.041717,-0.095383
3,0.73359,0.687126,2.058055,0.471056,0.966678
5,-0.96717,-0.302166,-0.794174,-1.434897,-0.318416


In [ ]:
input_size = X_train.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,1)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NeuralNetwork().to(device)


criteria = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epoch = 10000

for epochs in range(epoch):
  model.train()
  optimizer.zero_grad()

  output = model(X_train_tensor)
  loss = criteria(output, y_train_tensor)
  loss.backward()
  optimizer.step()

  if((epochs+1)%500 ==0):
    print(f"Epoch: {epochs+1}, Loss: {loss.item()}")



Epoch: 500, Loss: 0.23271939158439636
Epoch: 1000, Loss: 0.22258597612380981
Epoch: 1500, Loss: 0.215470552444458
Epoch: 2000, Loss: 0.2095026671886444
Epoch: 2500, Loss: 0.20258210599422455
Epoch: 3000, Loss: 0.19833171367645264
Epoch: 3500, Loss: 0.19331994652748108
Epoch: 4000, Loss: 0.1900993287563324
Epoch: 4500, Loss: 0.18724852800369263
Epoch: 5000, Loss: 0.18553172051906586
Epoch: 5500, Loss: 0.18308474123477936
Epoch: 6000, Loss: 0.18138636648654938
Epoch: 6500, Loss: 0.1796572059392929
Epoch: 7000, Loss: 0.17812371253967285
Epoch: 7500, Loss: 0.17795179784297943
Epoch: 8000, Loss: 0.17655421793460846
Epoch: 8500, Loss: 0.1750970184803009
Epoch: 9000, Loss: 0.17347201704978943
Epoch: 9500, Loss: 0.1743299514055252
Epoch: 10000, Loss: 0.1738814413547516


In [ ]:
model.eval()
with torch.no_grad():
    test_output = model(X_test_tensor)
    test_loss = criteria(test_output, y_test_tensor)

    test_output_actual = (test_output * pricestd) + pricemean
    y_test_actual = (y_test_tensor * pricestd) + pricemean

for i in range(5):  # Print first 5 examples
    print(f"Predicted Price: {test_output_actual[i].item():,.2f}, Actual Price: {y_test_actual[i].item():,.2f}")
    mse = criteria(test_output_actual[i], y_test_actual[i])

    mse = F.mse_loss(test_output_actual, y_test_actual)
    rmse = torch.sqrt(mse)

    mape = torch.mean(torch.abs((test_output_actual - y_test_actual) / y_test_actual))

    accuracy = 1 - mape


print(f"Test Loss: {test_loss.item()}")
print(accuracy.item()*100,"%")



Predicted Price: 909,719.12, Actual Price: 805,000.00
Predicted Price: 951,211.38, Actual Price: 1,070,000.00
Predicted Price: 1,746,719.88, Actual Price: 1,750,000.00
Predicted Price: 1,066,442.75, Actual Price: 949,000.00
Predicted Price: 882,948.75, Actual Price: 865,000.00
Test Loss: 0.3356108069419861
75.12199878692627 %


In [ ]:
criteria1 = nn.MSELoss()
optimizer1 = optim.Adam(model.parameters(), lr=0.001)

model1 = NeuralNetwork().to(device)

epoch = 10000

for epochs in range(epoch):
  model1.train()
  optimizer1.zero_grad()

  output = model1(X_train_tensor)
  loss = criteria1(output, y_train_tensor)
  loss.backward()
  optimizer1.step()

  if((epochs+1)%500 ==0):
    print(f"Epoch: {epochs+1}, Loss: {loss.item()}")



Epoch: 500, Loss: 0.7449046969413757
Epoch: 1000, Loss: 0.7449046969413757
Epoch: 1500, Loss: 0.7449046969413757
Epoch: 2000, Loss: 0.7449046969413757
Epoch: 2500, Loss: 0.7449046969413757
Epoch: 3000, Loss: 0.7449046969413757
Epoch: 3500, Loss: 0.7449046969413757
Epoch: 4000, Loss: 0.7449046969413757
Epoch: 4500, Loss: 0.7449046969413757
Epoch: 5000, Loss: 0.7449046969413757
Epoch: 5500, Loss: 0.7449046969413757
Epoch: 6000, Loss: 0.7449046969413757
Epoch: 6500, Loss: 0.7449046969413757
Epoch: 7000, Loss: 0.7449046969413757
Epoch: 7500, Loss: 0.7449046969413757
Epoch: 8000, Loss: 0.7449046969413757
Epoch: 8500, Loss: 0.7449046969413757
Epoch: 9000, Loss: 0.7449046969413757
Epoch: 9500, Loss: 0.7449046969413757
Epoch: 10000, Loss: 0.7449046969413757


In [ ]:
model1.eval()
with torch.inference_mode():
    test_output = model1(X_test_tensor)
    test_loss = criteria(test_output, y_test_tensor)

    test_output_actual = (test_output * pricestd) + pricemean
    y_test_actual = (y_test_tensor * pricestd) + pricemean

for i in range(5):  # Print first 5 examples
    print(f"Predicted Price: {test_output_actual[i].item():,.2f}, Actual Price: {y_test_actual[i].item():,.2f}")
    mse = criteria1(test_output_actual[i], y_test_actual[i])

    mse = F.mse_loss(test_output_actual, y_test_actual)
    rmse = torch.sqrt(mse)

    mape = torch.mean(torch.abs((test_output_actual - y_test_actual) / y_test_actual))

    accuracy = 1 - mape


print(f"Test Loss: {test_loss.item()}")
print(accuracy.item()*100,"%")



Predicted Price: 1,072,270.50, Actual Price: 805,000.00
Predicted Price: 1,125,612.88, Actual Price: 1,070,000.00
Predicted Price: 1,088,773.75, Actual Price: 1,750,000.00
Predicted Price: 1,049,425.75, Actual Price: 949,000.00
Predicted Price: 1,071,900.75, Actual Price: 865,000.00
Test Loss: 1.9098358154296875
54.06808853149414 %
